Get the raw data from https://www.kaggle.com/datasets/paololol/league-of-legends-ranked-matches/data?select=stats1.csv

The data obtained from Kaggle is already cleaned, so further cleaning is unnecessary.

Due to the large dataset size, the data is divided into two files. We concatenate these files and then merge them with participant data and match data. 
After merging, we filter the data to include only matches with exactly 10 participants, ensuring that roles are not duplicated within each team.

In [2]:
import pandas as pd
import numpy as np

# Load the data
participant_data = pd.read_csv('./data/raw/participants.csv')
stat1_data = pd.read_csv('./data/raw/stats1.csv')
stat2_data = pd.read_csv('./data/raw/stats2.csv')
matches_data = pd.read_csv('./data/raw/matches.csv')

# Concatenate stat1 and stat2
merged_stat_data = pd.concat([stat1_data, stat2_data])

# Step 1: Merge participant_data with merged_stat_data on 'id'
merged_data = pd.merge(participant_data, merged_stat_data, on='id', how='outer')

# Step 2: Merge the resulting data with matches_data on 'matchid' and 'id'
merged_data = pd.merge(merged_data, matches_data, left_on='matchid', right_on='id', how='inner')

# Step 3: Remove the 'id_y' column and rename 'id_x' to 'participantid'
merged_data = merged_data.drop(columns=['id_y']).rename(columns={'id_x': 'participantid'})

# Step 4: Count the occurrences of each match_id in the merged_data
match_id_counts = merged_data['matchid'].value_counts()

# Step 5: Find match_ids that appear exactly 10 times
match_ids_with_10_records = match_id_counts[match_id_counts == 10].index

# Step 6: Filter merged_data to include only those records where the match_id appears 10 times
participants_with_10_records = merged_data[merged_data['matchid'].isin(match_ids_with_10_records)]

# Step 7: Define conditions for assigning roles
conditions = [
    (participants_with_10_records['role'] == 'SOLO') & (participants_with_10_records['position'] == 'TOP'),
    (participants_with_10_records['role'] == 'NONE') & (participants_with_10_records['position'] == 'JUNGLE'),
    (participants_with_10_records['role'] == 'SOLO') & (participants_with_10_records['position'] == 'MID'),
    (participants_with_10_records['role'] == 'DUO_CARRY') & (participants_with_10_records['position'] == 'BOT'),
    (participants_with_10_records['role'] == 'DUO_SUPPORT') & (participants_with_10_records['position'] == 'BOT')
]

# Define the corresponding values for each condition
values = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']

# Create a new column 'role_position' and apply the conditions
participants_with_10_records['role_position'] = np.select(conditions, values, default='UNKNOWN')

# Step 8: Check each match_id to ensure both teams have exactly 5 unique roles
valid_matches = []

# Loop through each match_id
for match_id in match_ids_with_10_records:
    # Get the participants for this match
    match_participants = participants_with_10_records[participants_with_10_records['matchid'] == match_id]
    
    # Split participants by their win/loss (1 = win, 0 = loss)
    team1 = match_participants[match_participants['win'] == 1]
    team2 = match_participants[match_participants['win'] == 0]
    
    # Ensure both teams have exactly 5 players and 5 unique roles
    if len(team1) == 5 and len(team2) == 5:
        if len(team1['role_position'].unique()) == 5 and len(team2['role_position'].unique()) == 5:
            valid_matches.append(match_id)

# Step 9: Filter the merged_data again to keep only valid matches
valid_participants = participants_with_10_records[participants_with_10_records['matchid'].isin(valid_matches)]

# Step 10: Save the result to a new CSV file
# valid_participants.to_csv('../data/new/parti10records_unique_role.csv', index=False)

print(f"Number of valid matches with exactly 5 unique roles per team based on win/loss: {len(valid_matches)}")


/var/folders/dw/x19hnlxd69lg2jy91nlhkxrm0000gn/T/ipykernel_65013/852831567.py:7: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  stat2_data = pd.read_csv('./data/raw/stats2.csv')
/var/folders/dw/x19hnlxd69lg2jy91nlhkxrm0000gn/T/ipykernel_65013/852831567.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  participants_with_10_records['role_position'] = np.select(conditions, values, default='UNKNOWN')


KeyboardInterrupt: 

Filter the columns to reduce the dataset size while retaining an overview of key metrics for each role.

In [ ]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('../data/new/sample_participants_70percent.csv')

# Specify the columns you want to keep
columns_to_keep = [
    'participantid', 'matchid', 'championid', 'win', 'kills', 'deaths', 'assists', 'largestkillingspree', 'largestmultikill', 
    'killingsprees', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'legendarykills',
    'totdmgdealt', 'totdmgtochamp', 'dmgtoobj', 'visionscore', 'totdmgtaken', 'goldearned', 'inhibkills', 
    'totminionskilled', 'neutralminionskilled', 'wardsbought', 'wardsplaced', 'wardskilled', 'role_position', 'duration'
]

# Filter the data
filtered_data = data[columns_to_keep]

# Save the filtered data to a new CSV file
output_file_path = '../data/new/filtered_sample_participants_70percent.csv'
filtered_data.to_csv(output_file_path, index=False)

Separate the dataset into subsets based on each role.